Automatic prediction of gender from face images has drawn a lot of attention recently, due to its wide application in various facial analysis problems. However, due to the large variations of face images (such as variation in lighting, scale, and occlusion) the existing models are still behind the desired accuracy level which is necessary for exploiting these models in real-world applications.

The goal of this tutorial is to develop a lightweight command-line-based utility, through Python-based modules to automatically detect faces in a static image and to predict the gender of the spotted persons using a deep learning-based gender detection model.

## Real time Age & Gender detection using OpenCV

###### Importing the libraries
Let’s start with importing the necessary libraries for the code

In [1]:
import cv2
import math
import time
import argparse

C:\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Python39\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


###### Create getFaceBox
We will create a function called getFaceBox for preprocessing the image ,create a blob of the image(so that it can concentrate on more important features) and detect the faces using pre-trained models such as opencv_face_detector_uint8.pb and if we detect a face(which we can only be detected if it has more probability than conf_threshold)we draw a box around it using cv2.rectangle() function and yes it will detect multiple face

In [2]:
def getFaceBox(net, frame,conf_threshold = 0.75):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn,1.0,(300,300),
                                 [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []

    for i in range(detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > conf_threshold:
            x1 = int(detections[0,0,i,3]* frameWidth)
            y1 = int(detections[0,0,i,4]* frameHeight)
            x2 = int(detections[0,0,i,5]* frameWidth)
            y2 = int(detections[0,0,i,6]* frameHeight)
            bboxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn,(x1,y1),(x2,y2),(0,255,0),
                          int(round(frameHeight/150)),8)

    return frameOpencvDnn , bboxes

###### Models and Variables
We are using caffe-models for age and gender detection and also defining variables for age and gender values

In [7]:
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

###### Load and Start
Now lets load the network by using cv2.dnn.readNet to load the pre-trained models of age,gender and face . Use the webcam of your system by using cv2.VideoCapture(0)

In [8]:
#load the network
ageNet = cv2.dnn.readNet(ageModel,ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
faceNet = cv2.dnn.readNet(faceModel, faceProto)

cap = cv2.VideoCapture(0)
padding = 20

###### Read, Resize and Detect
Now read the frame , resize it for better accuracy(come on not all of us have GPU’s ) detect the faces and boxes using getFaceBox function , and create another blob with size of (227,227) (it’s easier for the model to process the frame with this size) and use the pretrained models on the blob and print out the probabilities and use cv2.imshow() to see the result in real-time and vola! one AGE-Gender detector right out of the Codeven(terrible pun ,I know)

In [ ]:
while cv2.waitKey(1) < 0:
    #read frame
    t = time.time()
    hasFrame , frame = cap.read()

    if not hasFrame:
        cv2.waitKey()
        break
    #creating a smaller frame for better optimization
    small_frame = cv2.resize(frame,(0,0),fx = 0.5,fy = 0.5)

    frameFace ,bboxes = getFaceBox(faceNet,small_frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue
    for bbox in bboxes:
        face = small_frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),
                max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]
        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow("Age Gender Demo", frameFace)
       
    print("time : {:.3f}".format(time.time() - t))

Gender : Female, conf = 0.999
Age Output : [[6.4196511e-06 5.6439436e-05 6.7407363e-03 1.5580532e-01 7.7555925e-01
  6.1579876e-02 5.9417591e-05 1.9249848e-04]]
Age : (25-32), conf = 0.776
time : 0.556
Gender : Female, conf = 0.999
Age Output : [[5.1237917e-06 6.4324231e-05 3.9100130e-03 2.9920408e-01 6.5294403e-01
  4.3638200e-02 6.3608488e-05 1.7062073e-04]]
Age : (25-32), conf = 0.653
time : 0.161
Gender : Female, conf = 0.999
Age Output : [[4.8236448e-06 3.5648773e-05 1.7297948e-03 5.2823372e-02 9.2762834e-01
  1.7574482e-02 4.3306241e-05 1.6028495e-04]]
Age : (25-32), conf = 0.928
time : 0.098
Gender : Female, conf = 0.998
Age Output : [[3.0759832e-06 3.8320773e-05 2.2542968e-03 7.8299239e-02 9.0111953e-01
  1.8141219e-02 3.7378817e-05 1.0694324e-04]]
Age : (25-32), conf = 0.901
time : 0.095
Gender : Female, conf = 0.998
Age Output : [[3.0759832e-06 3.8320773e-05 2.2542968e-03 7.8299239e-02 9.0111953e-01
  1.8141219e-02 3.7378817e-05 1.0694324e-04]]
Age : (25-32), conf = 0.901
tim

Gender : Male, conf = 0.576
Age Output : [[1.3905636e-04 3.2973243e-04 4.0799459e-03 6.5068722e-02 2.3311859e-02
  3.7316772e-01 1.4787740e-01 3.8602552e-01]]
Age : (60-100), conf = 0.386
time : 0.147
Gender : Female, conf = 0.541
Age Output : [[9.2348957e-05 1.8946931e-04 4.4589927e-03 9.1986179e-02 4.3135196e-02
  6.2417465e-01 8.8057257e-02 1.4790584e-01]]
Age : (38-43), conf = 0.624
time : 0.128
Gender : Female, conf = 0.724
Age Output : [[0.00087383 0.00107907 0.00885741 0.16220346 0.03923264 0.30948988
  0.12919167 0.349072  ]]
Age : (60-100), conf = 0.349
time : 0.151
Gender : Female, conf = 0.507
Age Output : [[4.6059076e-04 1.6590671e-04 3.5727029e-03 2.3338582e-02 1.1045428e-02
  2.4187359e-01 2.4664502e-01 4.7289816e-01]]
Age : (60-100), conf = 0.473
time : 0.153
Gender : Male, conf = 0.585
Age Output : [[1.49966101e-04 2.40018213e-04 2.06280104e-03 6.03280067e-02
  5.43295182e-02 6.32050574e-01 2.35964343e-01 1.48748085e-02]]
Age : (38-43), conf = 0.632
time : 0.105
Gender 

Gender : Female, conf = 0.998
Age Output : [[9.0974694e-08 1.9482610e-07 9.9878380e-05 1.4099000e-03 9.9131763e-01
  7.0567844e-03 9.8540904e-06 1.0576286e-04]]
Age : (25-32), conf = 0.991
time : 0.086
Gender : Female, conf = 0.998
Age Output : [[7.8086572e-08 1.1701306e-07 3.3935012e-05 1.3326666e-03 9.9099815e-01
  7.5428337e-03 1.0858640e-05 8.1249360e-05]]
Age : (25-32), conf = 0.991
time : 0.108
Gender : Female, conf = 0.998
Age Output : [[3.1390538e-08 1.0428654e-07 7.0438524e-05 1.9470386e-03 9.9451315e-01
  3.4264757e-03 4.5320517e-06 3.8325990e-05]]
Age : (25-32), conf = 0.995
time : 0.135
Gender : Female, conf = 0.998
Age Output : [[1.0011533e-07 1.6578576e-07 4.8825070e-05 1.4572813e-03 9.9334311e-01
  5.0798114e-03 8.5316096e-06 6.2196152e-05]]
Age : (25-32), conf = 0.993
time : 0.088
Gender : Female, conf = 0.997
Age Output : [[2.1991123e-07 4.1066335e-07 6.7128174e-05 2.3246473e-03 9.8841757e-01
  8.9964466e-03 1.9656945e-05 1.7392490e-04]]
Age : (25-32), conf = 0.988
tim

Gender : Female, conf = 0.997
Age Output : [[6.2845523e-08 1.6272037e-06 2.0882427e-03 2.3183070e-02 9.6939051e-01
  5.2712206e-03 4.6675627e-06 6.0712096e-05]]
Age : (25-32), conf = 0.969
time : 0.104
Gender : Female, conf = 0.995
Age Output : [[2.1074134e-08 4.8380451e-07 9.9867582e-04 5.7495325e-03 9.9040389e-01
  2.7985822e-03 3.5630280e-06 4.5209705e-05]]
Age : (25-32), conf = 0.990
time : 0.081
Gender : Female, conf = 0.997
Age Output : [[5.9467887e-08 8.7097055e-07 4.7938965e-04 1.2489531e-02 9.8438895e-01
  2.4806501e-03 6.1251658e-06 1.5459533e-04]]
Age : (25-32), conf = 0.984
time : 0.092
Gender : Female, conf = 0.997
Age Output : [[1.4305617e-08 2.0885187e-07 4.3784062e-04 1.0778847e-02 9.8516786e-01
  3.5729108e-03 1.9918516e-06 4.0273964e-05]]
Age : (25-32), conf = 0.985
time : 0.094
Gender : Female, conf = 0.997
Age Output : [[8.1135909e-08 1.6847566e-06 1.1320532e-03 1.0402656e-02 9.8613203e-01
  2.2132343e-03 4.4112658e-06 1.1381677e-04]]
Age : (25-32), conf = 0.986
tim

Gender : Female, conf = 0.998
Age Output : [[1.01047142e-07 5.32810475e-07 1.80987653e-03 7.18911970e-03
  9.54731703e-01 3.62100787e-02 1.43539055e-05 4.42927412e-05]]
Age : (25-32), conf = 0.955
time : 0.093
Gender : Female, conf = 0.998
Age Output : [[5.8832711e-08 4.5584241e-07 1.4052236e-03 1.7061487e-02 9.5933771e-01
  2.2154085e-02 9.3299259e-06 3.1648087e-05]]
Age : (25-32), conf = 0.959
time : 0.142
Gender : Female, conf = 0.997
Age Output : [[5.9305787e-08 2.3455627e-07 4.5260615e-04 6.0773804e-03 9.7942388e-01
  1.4021011e-02 8.4656349e-06 1.6255777e-05]]
Age : (25-32), conf = 0.979
time : 0.105
Gender : Female, conf = 0.997
Age Output : [[8.2471090e-08 8.8328534e-07 2.5294381e-03 1.9989448e-02 9.5291132e-01
  2.4505259e-02 1.2192842e-05 5.1486750e-05]]
Age : (25-32), conf = 0.953
time : 0.161
Gender : Female, conf = 0.995
Age Output : [[4.8011234e-08 7.3318233e-08 1.6861891e-04 1.1433960e-03 9.5701975e-01
  4.1630171e-02 1.3254773e-05 2.4712690e-05]]
Age : (25-32), conf = 0

...

time : 0.178
Gender : Female, conf = 0.982
Age Output : [[3.3305739e-06 2.2368439e-04 2.5267163e-01 1.1646958e-01 6.0274553e-01
  2.4895513e-02 4.8630234e-04 2.5044114e-03]]
Age : (25-32), conf = 0.603
time : 0.108
Gender : Female, conf = 0.960
Age Output : [[3.7502741e-06 3.2568877e-04 2.0768525e-01 2.0391691e-01 5.6005186e-01
  2.4585778e-02 6.3905533e-04 2.7916816e-03]]
Age : (25-32), conf = 0.560
time : 0.107
Gender : Female, conf = 0.934
Age Output : [[1.9328374e-05 3.5123059e-03 1.9306415e-01 3.8711432e-01 4.0301469e-01
  1.0566803e-02 5.3285039e-04 2.1754419e-03]]
Age : (25-32), conf = 0.403
time : 0.173
Gender : Female, conf = 0.953
Age Output : [[1.2175515e-05 6.6954928e-04 1.5210807e-01 4.0878290e-01 3.8759524e-01
  3.7572343e-02 3.1587873e-03 1.0100824e-02]]
Age : (15-20), conf = 0.409
time : 0.129
Gender : Female, conf = 0.830
Age Output : [[1.6736978e-05 2.2488768e-04 5.0649166e-02 1.7206368e-01 7.1196264e-01
  4.0787067e-02 7.9876455e-03 1.6308255e-02]]
Age : (25-32), con

Gender : Female, conf = 0.984
Age Output : [[7.3621123e-08 1.6024548e-07 8.8455155e-04 1.2232129e-03 9.9495983e-01
  2.8832874e-03 1.0764025e-05 3.8059297e-05]]
Age : (25-32), conf = 0.995
time : 0.117
Gender : Female, conf = 0.964
Age Output : [[1.9139112e-07 9.0943041e-07 1.5436377e-03 7.2174491e-03 9.8773789e-01
  3.3841215e-03 1.5708012e-05 1.0021458e-04]]
Age : (25-32), conf = 0.988
time : 0.183
Gender : Female, conf = 0.970
Age Output : [[1.3538966e-07 4.2656174e-07 2.2717549e-03 1.7183728e-03 9.9240571e-01
  3.5155341e-03 2.2311897e-05 6.5643311e-05]]
Age : (25-32), conf = 0.992
time : 0.110
Gender : Female, conf = 0.889
Age Output : [[8.4189750e-08 2.6952335e-07 1.1811731e-03 2.4575833e-03 9.9315989e-01
  3.1426104e-03 1.1886380e-05 4.6553836e-05]]
Age : (25-32), conf = 0.993
time : 0.151
Gender : Female, conf = 0.976
Age Output : [[6.1275010e-08 5.1151824e-07 5.0524911e-03 3.9811283e-03 9.8867512e-01
  2.2457547e-03 1.0343154e-05 3.4504970e-05]]
Age : (25-32), conf = 0.989
tim

time : 0.123
Gender : Female, conf = 0.987
Age Output : [[2.7723402e-06 9.7728880e-06 4.1523907e-02 1.9052939e-02 8.8842785e-01
  4.9765091e-02 5.3359987e-04 6.8406097e-04]]
Age : (25-32), conf = 0.888
time : 0.154
Gender : Female, conf = 0.996
Age Output : [[3.0786842e-07 1.6705494e-06 1.4706258e-02 1.0889656e-02 9.3559420e-01
  3.8375124e-02 1.0089351e-04 3.3179848e-04]]
Age : (25-32), conf = 0.936
time : 0.176
Gender : Female, conf = 0.995
Age Output : [[1.3054604e-05 3.7157202e-05 6.3987519e-03 1.1494623e-02 8.9435738e-01
  2.3646129e-02 8.2837086e-04 6.3224539e-02]]
Age : (25-32), conf = 0.894
time : 0.180
Gender : Female, conf = 0.990
Age Output : [[1.4905393e-05 1.0251908e-04 7.9579391e-03 2.6671776e-01 6.9216228e-01
  1.4654788e-02 2.6503054e-04 1.8124824e-02]]
Age : (25-32), conf = 0.692
time : 0.112
Gender : Female, conf = 0.848
Age Output : [[8.4747560e-05 1.7697320e-04 2.9511321e-03 1.9717564e-01 6.4656228e-01
  2.3857547e-02 2.0848576e-03 1.2710685e-01]]
Age : (25-32), con

Gender : Female, conf = 0.994
Age Output : [[4.4851195e-06 3.3143391e-05 6.6915229e-03 7.3948251e-03 9.6635681e-01
  1.0144128e-02 1.2141467e-04 9.2537152e-03]]
Age : (25-32), conf = 0.966
time : 0.158
Gender : Female, conf = 0.992
Age Output : [[2.5626889e-06 2.0507427e-05 9.4989371e-03 8.1211440e-03 9.5151156e-01
  2.4619292e-02 2.2867734e-04 5.9972429e-03]]
Age : (25-32), conf = 0.952
time : 0.136
Gender : Female, conf = 0.995
Age Output : [[1.1182162e-06 6.3520533e-06 2.1377394e-03 3.8841399e-03 9.8242891e-01
  1.0376365e-02 5.6641053e-05 1.1087749e-03]]
Age : (25-32), conf = 0.982
time : 0.164
Gender : Female, conf = 0.991
Age Output : [[4.2153001e-06 1.1494974e-05 1.6190960e-03 1.9077968e-03 9.6681756e-01
  1.2466794e-02 5.8748736e-04 1.6585670e-02]]
Age : (25-32), conf = 0.967
time : 0.110
Gender : Female, conf = 0.995
Age Output : [[8.82465520e-06 1.81910422e-04 1.20875426e-01 2.73494720e-01
  5.78253746e-01 7.01816892e-03 3.61721002e-04 1.98054723e-02]]
Age : (25-32), conf = 0

Gender : Female, conf = 0.743
Age Output : [[1.2838298e-05 6.3186773e-05 1.6694535e-02 4.2614836e-01 4.9786383e-01
  5.3783569e-02 3.7147859e-03 1.7189130e-03]]
Age : (25-32), conf = 0.498
time : 0.133
Gender : Female, conf = 0.830
Age Output : [[1.9574509e-06 1.6359680e-05 9.7364346e-03 7.3230672e-01 2.1689019e-01
  4.0118802e-02 5.9441396e-04 3.3514781e-04]]
Age : (15-20), conf = 0.732
time : 0.133
Gender : Female, conf = 0.898
Age Output : [[3.4794255e-06 1.5400925e-05 7.2350637e-03 5.0326252e-01 4.3060631e-01
  5.8179148e-02 3.2808766e-04 3.6999959e-04]]
Age : (15-20), conf = 0.503
time : 0.146
Gender : Female, conf = 0.932
Age Output : [[7.1487275e-06 2.2739068e-05 6.3530821e-03 2.5229150e-01 6.8544424e-01
  5.4052453e-02 1.1016426e-03 7.2725612e-04]]
Age : (25-32), conf = 0.685
time : 0.134
Gender : Female, conf = 0.950
Age Output : [[5.9199010e-06 2.0784504e-05 3.4969358e-03 5.2252239e-01 4.3612331e-01
  3.6627188e-02 5.0034310e-04 7.0314645e-04]]
Age : (15-20), conf = 0.523
tim